In [ ]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
# from mmcv.runner.hooks import HOOKS, Hook
#import wandb

In [ ]:
# @HOOKS.register_module()
# class ImageDetection(Hook):
#     """Check invalid loss hook.
#     This hook will regularly check whether the loss is valid
#     during training.
#     Args:
#         interval (int): Checking interval (every k iterations).
#             Default: 50.
#     """

#     def __init__(self, interval=50):
#         self.interval = interval
        
#     def after_train_epoch(self, runner):
#         print(dir(runner))

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/retinanet/retinanet_r18_fpn_1x_coco_cosineAnn.py')
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'stratified_v2/train_fold_1.json' # train json 정보
#cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'stratified_v2/val_fold_1.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

# cfg.train_pipeline = cfg.train_pipeline
# cfg.val_pipeline = cfg.test_pipeline
# cfg.test_pipeline = cfg.test_pipeline

# cfg.data.train.pipeline = cfg.train_pipeline
# cfg.data.val.pipeline = cfg.val_pipeline
# cfg.data.test.pipeline = cfg.test_pipeline

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/kfold5_retinanet_r18_trash'

#cfg.evaluation = dict(interval=1, start=1,save_best='auto')

#faster_rcnn : resnet50, resnet101 계열을 classes 지정해줘야 오류가 안 생김
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = None #dict(max_norm=35, norm_type=2)
# 옵티마이저 lr 수정해 볼 수 있음
#cfg.optimizer.lr=0.000005
#cfg.lr_config.step=[4]
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='Model', name='RetinaNet+ResNet18', entity='hi-ai')
            )])
# for wandb
cfg.log_config = log_config

cfg.device = get_device()
cfg.runner = dict(type='EpochBasedRunner', max_epochs=12)

In [ ]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

In [ ]:
# dataset 확인
datasets[0]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [ ]:
# meta = dict()
# meta['fp16'] = dict(loss_scale=dict(init_scale=512))

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)
#train_detector(model, datasets, cfg, distributed=False, validate=True)
#train_detector(model, datasets, cfg, distributed=False, validate=True, meta=meta)